So far, this is just a proof of concept for using the API. Obviously, we need to streamline the process of pulling more rows, loading/saving the csv as well as cleaning the data. 

In [2]:
%pip install requests keyring

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cdg_client import CDGClient
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd

In [54]:
API_KEY_PATH = '../../../api_key.txt'       # put your api_key path here
                                            # it shouldn't go to github
LIMIT = 250 
parse_xml = lambda data: ET.fromstring(data)


with open(API_KEY_PATH) as file:
    api_key = file.readline()
client = CDGClient(api_key, response_format='xml')
data, _ = client.get(f'bill?format=xml&limit={100}&fromDateTime=2023-01-01T00:00:00Z&toDateTime=2023-03-11T23:59:59Z')

In [5]:
bills_dict = xmltodict.parse(data)['api-root']['bills']['bill']
len(bills_dict)

100

In [3]:
df = pd.read_csv('congress_bills.csv')

In [5]:
len(df[df.became_law == True])/len(df)

0.020412811387900357

In [52]:
url = df.iloc[0]['url'][28:]
id = url[5:-11].replace('/','-')
id

'117-s-5320'

In [20]:
pd.DataFrame({'action_date': [pd.NaT]})

,action_date
0,NaT


In [53]:
data, _ = client.get('bill/119/hr/1/actions?format=xml')
xmltodict.parse(data)['api-root']

{'actions': {'item': [{'actionDate': '2025-07-04',
    'text': 'Became Public Law No: 119-21.',
    'type': 'President',
    'actionCode': 'E40000',
    'sourceSystem': {'code': '9', 'name': 'Library of Congress'}},
   {'actionDate': '2025-07-04',
    'text': 'Became Public Law No: 119-21.',
    'type': 'BecameLaw',
    'actionCode': '36000',
    'sourceSystem': {'code': '9', 'name': 'Library of Congress'}},
   {'actionDate': '2025-07-04',
    'text': 'Signed by President.',
    'type': 'President',
    'actionCode': 'E30000',
    'sourceSystem': {'code': '9', 'name': 'Library of Congress'}},
   {'actionDate': '2025-07-04',
    'text': 'Signed by President.',
    'type': 'BecameLaw',
    'actionCode': '36000',
    'sourceSystem': {'code': '9', 'name': 'Library of Congress'}},
   {'actionDate': '2025-07-03',
    'text': 'Presented to President.',
    'type': 'Floor',
    'actionCode': 'E20000',
    'sourceSystem': {'code': '2', 'name': 'House floor actions'}},
   {'actionDate': '2025-07

In [43]:
data, _ = client.get('bill/119/s/2403/actions?format=xml')
pd.DataFrame(xmltodict.parse(data)['api-root']['actions']['item'])

,actionDate,text,type,sourceSystem,actionCode,committees
0,2025-10-09,Passed Senate with an amendment by Unanimous C...,Floor,{'name': 'Senate'},NaN,NaN
1,2025-10-09,Passed/agreed to in Senate: Passed Senate with...,Floor,"{'code': '9', 'name': 'Library of Congress'}",17000,NaN
2,2025-09-11,Placed on Senate Legislative Calendar under Ge...,Calendars,{'name': 'Senate'},NaN,NaN
3,2025-09-11,"Committee on Health, Education, Labor, and Pen...",Committee,{'name': 'Senate'},NaN,{'item': {'url': 'https://api.congress.gov/v3/...
4,2025-09-11,"Committee on Health, Education, Labor, and Pen...",Committee,"{'code': '9', 'name': 'Library of Congress'}",14000,{'item': {'url': 'https://api.congress.gov/v3/...
5,2025-07-30,"Committee on Health, Education, Labor, and Pen...",Committee,{'name': 'Senate'},NaN,{'item': {'url': 'https://api.congress.gov/v3/...
6,2025-07-23,Read twice and referred to the Committee on He...,IntroReferral,{'name': 'Senate'},NaN,{'item': {'url': 'https://api.congress.gov/v3/...
7,2025-07-23,Introduced in Senate,IntroReferral,"{'code': '9', 'name': 'Library of Congress'}",10000,NaN


In [9]:
df.iloc[1]

congress                                                                 117
type                                                                       S
originChamber                                                         Senate
originChamberCode                                                          S
number                                                                  5291
url                        https://api.congress.gov/v3/bill/117/s/5291?fo...
title                      A bill to delay the implementation of the modi...
updateDateIncludingText                                           2023-03-02
latestAction               {'actionDate': '2022-12-15', 'text': 'Read twi...
updateDate                                                        2023-03-02
Name: 1, dtype: object

In [10]:
# get the next LIMIT # of bills and add it to the data frame over and over
for i in range(1, 5):
    offset = i * LIMIT
    more_data, _ = client.get(f'bill?format=json&limit={LIMIT}&offset={offset}') 
    more_bills = xmltodict.parse(more_data)['api-root']['bills']['bill']
    more_rows = pd.DataFrame(more_bills)
    df = pd.concat([df, more_rows], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   congress                 1100 non-null   object
 1   type                     1100 non-null   object
 2   originChamber            1100 non-null   object
 3   originChamberCode        1100 non-null   object
 4   number                   1100 non-null   object
 5   url                      1100 non-null   object
 6   title                    1100 non-null   object
 7   updateDateIncludingText  1100 non-null   object
 8   latestAction             1100 non-null   object
 9   updateDate               1100 non-null   object
dtypes: object(10)
memory usage: 86.1+ KB


In [11]:
# save as csv 
# # probably want to go to dropbox or something, not directly in the repository
df.to_csv('congress_bills.csv') 